In [1]:
import pandas as pd
df1 = pd.read_csv(r"D:\临床数据\NHANES数据清洗\3年心因死亡数据.csv", low_memory=False)
selected_columns = [
    "SEQN","Doctor told you have diabetes", "Education Level - Adults 20+", "Ever told you had a stroke",
    "Ever told you had coronary heart disease", "Ever told you had high blood pressure",
    "Had at least 12 alcohol drinks/1 yr?", "Marital Status", "Moderate recreational activities",
    "Moderate work activity", "Race", "Smoked at least 100 cigarettes in life", 
    "Vigorous recreational activities", "Vigorous work activity", "Walk or bicycle", "Age", 
    "Minutes sedentary activity", "Ratio of family income to poverty", "Waist Circumference (cm)", 
    "Alanine Aminotransferase (ALT) (U/L)", "Albumin (g/L)", "Albumin_urine (mg/L)", 
    "Alkaline Phosphatase (ALP) (IU/L)", "Bicarbonate (mmol/L)", "Blood urea nitrogen (mmol/L)", 
    "Chloride (mmol/L)", "Cholesterol (mmol/L)", "Creatinine (umol/L)", "Creatinine_urine (umol/L)", 
    "Direct HDL-Cholesterol (mmol/L)", "Gamma Glutamyl Transferase (GGT) (U/L)", "Globulin (g/L)", 
    "Glucose_serum (mmol/L)", "Glycohemoglobin (%)", "Hematocrit (%)", "Hemoglobin (g/dL)", 
    "Iron_refigerated (umol/L)", "Lactate Dehydrogenase (LDH) (U/L)", "Lymphocyte number (1000 cells/uL)", 
    "Lymphocyte percent (%)", "Mean cell hemoglobin (pg)", "Mean cell hemoglobin concentration (g/dL)", 
    "Mean cell volume (fL)", "Monocyte number (1000 cells/uL)", "Monocyte percent (%)", 
    "Osmolality (mmol/Kg)", "Phosphorus (mmol/L)", "Platelet count (1000 cells/uL)", 
    "Potassium (mmol/L)", "Red blood cell count (million cells/uL)", "Red cell distribution width (%)", 
    "Segmented neutrophils num (1000 cell/uL)", "Segmented neutrophils percent (%)", 
    "Total protein (g/L)", "Uric acid (umol/L)", "3 year heart death"
]
df2 = df1[selected_columns]

In [3]:
from sklearn.preprocessing import StandardScaler
first_col = df2.iloc[:, 0]
last_col = df2.iloc[:, -1]
for col in df2.columns[1:15]:
    df2[col] = df2[col].astype('category')
df_encoded = pd.get_dummies(df2.iloc[:, 1:15], drop_first=True)
scaler = StandardScaler()
df_standardized = pd.DataFrame(scaler.fit_transform(df2.iloc[:, 15:-1]), columns=df2.columns[15:-1])
df3 = pd.concat([first_col, df_encoded, df_standardized, last_col], axis=1)

C:\Users\wzc\AppData\Local\Temp\ipykernel_22852\1171126219.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2[col] = df2[col].astype('category')


In [4]:
selected_columns = [
    "SEQN",'Age', 'Ratio of family income to poverty', 'Alanine Aminotransferase (ALT) (U/L)', 'Race_2','Race_3','Race_4','Race_5', 
    'Chloride (mmol/L)',
    'Minutes sedentary activity', 'Walk or bicycle_2', 'Glucose_serum (mmol/L)', 'Direct HDL-Cholesterol (mmol/L)', 
    'Mean cell volume (fL)', 
    "3 year heart death"
]
df4 = df3[selected_columns]

In [6]:
x = df4.iloc[:, 1:14]
y = df4['3 year heart death']
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)
from sklearn.linear_model import LogisticRegression
estimator = LogisticRegression(max_iter=200)
estimator.fit(x_train, y_train)
from sklearn.metrics import roc_auc_score
y_true = y_test
probs1 = estimator.predict_proba(x_test)[:, 1]
# 计算AUC
auc_LR = roc_auc_score(y_true, probs1)
auc_LR

0.8841694001629651

In [1]:
import numpy as np
from sklearn.metrics import roc_auc_score, roc_curve, confusion_matrix, accuracy_score, precision_recall_curve, f1_score, matthews_corrcoef
from sklearn.utils import resample
from sklearn.metrics import precision_score, recall_score, accuracy_score
y_probs = estimator.predict_proba(x_test)[:, 1]
auc = roc_auc_score(y_test, y_probs)
n_iterations = 1000
auc_scores = []
for i in range(n_iterations):
    x_resampled, y_resampled = resample(x_test, y_test, random_state=i)
    y_probs_resampled = estimator.predict_proba(x_resampled)[:, 1]
    auc_scores.append(roc_auc_score(y_resampled, y_probs_resampled))
auc_lower = np.percentile(auc_scores, 2.5)
auc_upper = np.percentile(auc_scores, 97.5)
fpr, tpr, thresholds = roc_curve(y_test, y_probs)
youden_index = tpr - fpr
best_threshold = thresholds[np.argmax(youden_index)]
y_pred_best = (y_probs >= best_threshold).astype(int)
cm = confusion_matrix(y_test, y_pred_best)
accuracy = accuracy_score(y_test, y_pred_best)
sensitivity = recall_score(y_test, y_pred_best)
specificity = cm[0, 0] / (cm[0, 0] + cm[0, 1])
ppv = precision_score(y_test, y_pred_best)
npv = cm[1, 1] / (cm[1, 1] + cm[1, 0])
mcc = matthews_corrcoef(y_test, y_pred_best)
f1 = f1_score(y_test, y_pred_best)
acc_scores, sn_scores, sp_scores, ppv_scores, npv_scores, mcc_scores, f1_scores = [], [], [], [], [], [], []
for i in range(n_iterations):
    x_resampled, y_resampled = resample(x_test, y_test, random_state=i)
    y_probs_resampled = estimator.predict_proba(x_resampled)[:, 1]
    y_pred_resampled = (y_probs_resampled >= best_threshold).astype(int)
    cm_resampled = confusion_matrix(y_resampled, y_pred_resampled)
    acc_scores.append(accuracy_score(y_resampled, y_pred_resampled))
    sn_scores.append(recall_score(y_resampled, y_pred_resampled))
    sp_scores.append(cm_resampled[0, 0] / (cm_resampled[0, 0] + cm_resampled[0, 1]))
    ppv_scores.append(precision_score(y_resampled, y_pred_resampled))
    npv_scores.append(cm_resampled[1, 1] / (cm_resampled[1, 1] + cm_resampled[1, 0]))
    mcc_scores.append(matthews_corrcoef(y_resampled, y_pred_resampled))
    f1_scores.append(f1_score(y_resampled, y_pred_resampled))
def calculate_ci(scores):
    return np.percentile(scores, 2.5), np.percentile(scores, 97.5)
accuracy_ci = calculate_ci(acc_scores)
sensitivity_ci = calculate_ci(sn_scores)
specificity_ci = calculate_ci(sp_scores)
ppv_ci = calculate_ci(ppv_scores)
npv_ci = calculate_ci(npv_scores)
mcc_ci = calculate_ci(mcc_scores)
f1_ci = calculate_ci(f1_scores)
print(f"AUC: {auc:.3f} ({auc_lower:.3f} - {auc_upper:.3f})")
print(f"Accuracy: {accuracy:.3f} ({accuracy_ci[0]:.3f} - {accuracy_ci[1]:.3f})")
print(f"Sensitivity: {sensitivity:.3f} ({sensitivity_ci[0]:.3f} - {sensitivity_ci[1]:.3f})")
print(f"Specificity: {specificity:.3f} ({specificity_ci[0]:.3f} - {specificity_ci[1]:.3f})")
print(f"PPV: {ppv:.3f} ({ppv_ci[0]:.3f} - {ppv_ci[1]:.3f})")
print(f"NPV: {npv:.3f} ({npv_ci[0]:.3f} - {npv_ci[1]:.3f})")
print(f"MCC: {mcc:.3f} ({mcc_ci[0]:.3f} - {mcc_ci[1]:.3f})")
print(f"F1 Score: {f1:.3f} ({f1_ci[0]:.3f} - {f1_ci[1]:.3f})")

NameError: name 'estimator' is not defined